In [0]:
from google.colab import drive


drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/ggcolab/'  #change dir to your project folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! pip install mpi4py

In [0]:
%%file hello.py

from mpi4py import MPI
comm = MPI.COMM_WORLD

print("Rank: "+ str(comm.rank))
print("Size: "+ str(comm.size))

Overwriting hello.py


In [0]:
! mpiexec -np 2 --allow-run-as-root python hello.py

Rank: 0
Rank: 1
Size: 2
Size: 2


In [0]:
%%file condition.py

from mpi4py import MPI
comm = MPI.COMM_WORLD

print("my rank is ", comm.rank)

if comm.rank == 1:
  print("doing task of rank 1")

if comm.rank == 0:
  print("doing task of rank 0")

if comm.rank == 2:
  print("doing task of rank 2")

Writing condition.py


In [0]:
! mpiexec -np 3 --allow-run-as-root python condition.py

my rank is  0
doing task of rank 0
my rank is  2
doing task of rank 2
my rank is  1
doing task of rank 1


In [0]:
%%file share.py

from mpi4py import MPI
import time

comm = MPI.COMM_WORLD

rank = comm.rank
size = comm.size
name = MPI.Get_processor_name()
message = "Hello"

if rank == 0:
  data = message
  comm.send(data, dest=1)
  print("from node", rank, "sent data")
  time.sleep(5)

elif comm.rank == 1:
  data = comm.recv(source=0)
  print("Node", rank, "received mess", data)

Overwriting share.py


In [0]:
! mpiexec -np 2 --allow-run-as-root python share.py

from node 0 sent data
Node 1 received mess Hello


In [0]:
%%file dynamic.py

from mpi4py import MPI
import time

comm = MPI.COMM_WORLD

rank = comm.rank
size = comm.size
name = MPI.Get_processor_name()
shared = (rank+1)%7

comm.send(shared,dest=(rank+1)%size)
data = comm.recv(source=(rank-1)%size)

print("Rank: ",rank)
print("Received: ",data, "coming from ",(rank-1)%size)

Overwriting dynamic.py


In [0]:
! mpiexec -np 5 --allow-run-as-root python dynamic.py

Rank:  3
Rank:  1
Received:  1 coming from  0
Rank:  2
Received:  2 coming from  1
Received:  3 coming from  2
Rank:  4
Received:  4 coming from  3
Rank:  0
Received:  5 coming from  4


In [0]:
%%file broadcast.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.rank

if rank == 0:
  data = {'a':1, "b":2, "c":3}

else:
  data = None

data = comm.bcast(data, root=0)
print("Rank", rank, data)

Overwriting broadcast.py


In [0]:
! mpiexec -np 6 --allow-run-as-root python broadcast.py

Rank 1 {'a': 1, 'b': 2, 'c': 3}
Rank 4 {'a': 1, 'b': 2, 'c': 3}
Rank 2 {'a': 1, 'b': 2, 'c': 3}
Rank 5 {'a': 1, 'b': 2, 'c': 3}
Rank 0 {'a': 1, 'b': 2, 'c': 3}
Rank 3 {'a': 1, 'b': 2, 'c': 3}


In [0]:
%%file request.py

import requests
from mpi4py import MPI
import time

urls = [
  "https://httpbin.org/ip",
  "https://httpbin.org/xml",
  "https://httpbin.org/json",
  "https://httpbin.org/image",
  "https://httpbin.org/image/png",
  "https://httpbin.org/image/jpg",
  "https://httpbin.org/jpeg",
  "https://httpbin.org/headers"
]


comm = MPI.COMM_WORLD

rank = comm.rank
size = comm.size
name = MPI.Get_processor_name()

if rank == 0:
  for i in [0,1,2,3]:
    r = requests.get(urls[i])
    print(r.content)
  print("Node", rank, "complete its job")

elif rank == 1:
  for i in [4,5,6,7]:
    r = requests.get(urls[i])
    print(r.content)
  print("Node", rank, "complete its job")

Overwriting request.py


In [0]:
! mpiexec -np 2 --allow-run-as-root python request.py

b'{\n  "origin": "35.247.24.83"\n}\n'
b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00d\x00\x00\x00d\x08\x02\x00\x00\x00\xff\x80\x02\x03\x00\x00\x1faIDATx\x9c\xdd}wXS\xd9\xd6\xfeJ#\xa1\x17\xc1 \x02\xa1\x08\xc1J\x19\x11E\x10\x1c\x11\x14\x10\xf8tl(\xea\xd8FGGT\xac\x9f\\\xeb\x8c\x0eV\xd4\xb1\xdca\xe4\n\xb6\x11\x1bEA\x14QA\x94&\x82\xa2B\x00EzoI \x10\x92\xf3\xfbc\x87\x10!\xc4$D\x7fw\xbe\xf7\xe1\xe199gg\xadu\xde\xb3\xf7>{\xaf\xbd\xd6\x0e\x0e\xc30\x00`\xb3\xd9\xc7\x8e\x1dKJJ\xca\xc9\xc9\xe9\xec\xec$\x93\xc9\x14\n\x85\xdc\x03UUU\x1a\x8dfjjjbbbjj\x8a\x0eTTT`\x10\xc00\xac\xbc\xbc\xbc\xb0\xb0\x90\xc1`0\x18\x8c\xf2\xf2r&\x93\xc9b\xb1\x98L&\x93\xc9\xe4\xf3\xf9\xea"000\xb0\xec\x81\x89\x89\t\x81@\x18\x8c\xea\xf6\xf6\xf6\xd2\xd2\xd2\x8f\x1f?~\xfc\xf8\x11\x1d|\xfa\xf4\x89\xcdfw\x8a\x80\xc3\xe1\x90\xc9d;;;77\xb7\xa0\xa0 UUU\x81\xd1\xa9\xa9\xa9&&&2\xe9\xc3\xe3\xf1vvvAAAw\xef\xdemmm\xc5\xa4Fyy\xf9\x993g\xa6O\x9f\xae\xac\xac,\xdf\xad*))9;;\x1f;v\xac\xa4\xa4Dz\xbd\xad\xad\xadw\xef\xde\r\n\n\xb2\xb3\xb3\xc3

In [0]:
%%file gather.py

import requests
from mpi4py import MPI

comm = MPI.COMM_WORLD

rank = comm.rank
size = comm.size
name = MPI.Get_processor_name()

sendbuf = []
root = 0
if comm.rank==0:
  urls = [
  "https://httpbin.org/ip",
  "https://httpbin.org/xml",
  "https://httpbin.org/json",
  "https://httpbin.org/image",
  ]
  sendbuf = urls

v = comm.scatter(sendbuf, root)
print("I got the url")
v = requests.get(v).content

recvbuf = comm.gather(v, root)
if comm.rank == 0:
  for i in recvbuf:
    print(i)


Overwriting gather.py


In [0]:
! mpiexec -np 4 --allow-run-as-root python gather.py

I got the url
I got the url
I got the url
I got the url
b'{\n  "origin": "35.247.24.83"\n}\n'
b'<?xml version=\'1.0\' encoding=\'us-ascii\'?>\n\n<!--  A SAMPLE set of slides  -->\n\n<slideshow \n    title="Sample Slide Show"\n    date="Date of publication"\n    author="Yours Truly"\n    >\n\n    <!-- TITLE SLIDE -->\n    <slide type="all">\n      <title>Wake up to WonderWidgets!</title>\n    </slide>\n\n    <!-- OVERVIEW -->\n    <slide type="all">\n        <title>Overview</title>\n        <item>Why <em>WonderWidgets</em> are great</item>\n        <item/>\n        <item>Who <em>buys</em> WonderWidgets</item>\n    </slide>\n\n</slideshow>'
b'{\n  "slideshow": {\n    "author": "Yours Truly", \n    "date": "date of publication", \n    "slides": [\n      {\n        "title": "Wake up to WonderWidgets!", \n        "type": "all"\n      }, \n      {\n        "items": [\n          "Why <em>WonderWidgets</em> are great", \n          "Who <em>buys</em> WonderWidgets"\n        ], \n        "title"